In [1]:
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import data_cleaning
from sklearn.model_selection import train_test_split
import numpy as np


from nltk.corpus import stopwords


df = pd.read_csv("/Users/lifeifan/Desktop/ece1786/project/data1.csv")


ModuleNotFoundError: No module named 'data_cleaning'

In [ ]:
cleaned_data = data_cleaning.remove_stop_words(df,"text","cleaned_text")
cleaned_data = data_cleaning.remove_punctuations(df,"cleaned_text","cleaned_text")

In [ ]:
df['cmp_code'] = df['cmp_code'].replace({'conservatism': 0, 'progressivism': 1})
df = df.rename(columns={'cmp_code': 'label'})
print(cleaned_data)

                                                   text  cmp_code  \
0     But too many Americans have been left out and ...         1   
1               and the racial wealth gap remains wide,         1   
2                empowerment is better than resentment,         1   
3                    and bridges are better than walls.         1   
4     —an economy that grows incomes for working peo...         1   
...                                                 ...       ...   
1995  To prevent sectors becoming partially or wholl...         0   
1996     We will promote integration and British values         0   
1997  Being able to speak English is a fundamental p...         0   
1998  We have introduced tough new language tests fo...         0   
1999  Next, we will legislate to ensure that every p...         0   

                                           cleaned_text  
0                   But many Americans left left behind  
1                        racial wealth gap remains wide

In [ ]:
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2LMHeadModel,
                          GPT2Tokenizer,
                          AdamW, 
                          TextDataset,
                          GPT2Config,
                          DataCollatorForLanguageModeling)

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, 
import torch
df_reset = df.reset_index(drop=True)
x_train, x_val, y_train, y_val = train_test_split(df['text'], df['label'], test_size=0.2, shuffle=True)
x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)



tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token



In [ ]:
def process_data(x,y):
    processed_data = []
    for i in range(len(x)):
        text = tokenizer(x[i], padding="max_length", truncation=True, return_tensors="pt")
        labels = y[i]
        processed_data.append({"input_ids": text["input_ids"], "attention_mask": text["attention_mask"], "labels": labels})
    return processed_data

train_data = process_data(x_train,y_train)

valid_data = process_data(x_val,y_val)

configuration = GPT2Config()
model = GPT2ForSequenceClassification(configuration).from_pretrained("gpt2",num_labels=2)
model.config.pad_token_id = model.config.eos_token_id


train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=16)



Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
label_to_id = model.config.label2id
print(label_to_id)

{'LABEL_0': 0, 'LABEL_1': 1}


In [ ]:
id_to_label = model.config.id2label
print(id_to_label)

{0: 'LABEL_0', 1: 'LABEL_1'}


In [ ]:
print('\nBert Model', model.summary())

In [12]:
from transformers import AdamW
import torch.nn.functional as F

train_loss_list = []
train_acc_list = []
learning_rate = 2e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
epochs = 10
for epoch in range(epochs):
    train_acc = 0
    train_loss = 0
    for batch in train_dataloader:
        inputs = batch["input_ids"].squeeze()
        attention_mask = batch["attention_mask"].squeeze()
        labels = batch["labels"]
        # print(inputs)
        # print(attention_mask)
        # print(labels)

        # Forward pass
        outputs = model(inputs, attention_mask=attention_mask, labels=labels)  
        # print(outputs)
        # Compute the loss
        loss = outputs.loss
        logit = F.softmax(outputs.logits, dim=1)
        train_pred = torch.round(logit).long()
        max_indices = torch.argmax(train_pred, dim=1)
        train_pred = torch.where(max_indices == 0, torch.tensor([0.0]), torch.tensor([1.0]))
        train_pred = train_pred.int()
        for i in range(len(labels)):
            if train_pred[i] == labels[i]:
                train_acc += 1
        train_loss += loss.item()
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
                
    train_loss /= len(train_dataloader)
    train_acc = train_acc/len(train_data)
    train_loss_list.append(train_loss)  
    train_acc_list.append(train_acc)

    # Print the loss after each epoch
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 